# IntelHealth - Colab Training (Per-Agent)

This notebook is designed for **one agent per run**.
Pick `AGENT_NAME`, then run all cells.

In [ ]:
# Clone repo + pull latest
import os
repo_dir = '/content/Intel_Health'
if not os.path.exists(repo_dir):
    !git clone https://github.com/DemonRain7/Intel_Health.git {repo_dir}
else:
    !git -C {repo_dir} pull
%cd {repo_dir}

In [ ]:
# Install dependencies
!pip -q install torch transformers datasets peft accelerate bitsandbytes sentencepiece loguru

## 1) Select Agent (single run)

Supported training agents:
- symptom_normalizer (SFT)
- symptom_quality_grader (SFT)
- rag_relevance_grader (SFT)
- drug_evidence_grader (SFT)
- diagnosis_generator (DAPT/QA)

In [ ]:
import subprocess, shlex
from pathlib import Path

AGENT_NAME = "symptom_quality_grader"  # change this each run

MODEL_BY_AGENT = {
    "symptom_normalizer": "Qwen/Qwen3-0.5B-Instruct",
    "symptom_quality_grader": "Qwen/Qwen3-0.5B-Instruct",
    "rag_relevance_grader": "Qwen/Qwen3-0.5B-Instruct",
    "drug_evidence_grader": "Qwen/Qwen3-0.5B-Instruct",
    "diagnosis_generator": "Qwen/Qwen3-1.8B-Instruct",
}

DATA_BY_AGENT = {
    "symptom_normalizer": "datasets/agent_sft/symptom_normalizer/preprocess_sft_data.jsonl",
    "symptom_quality_grader": "datasets/agent_sft/symptom_quality_grader/symptom_quality_grader_sft_data.jsonl",
    "rag_relevance_grader": "datasets/agent_sft/rag_relevance_grader/rag_relevance_grader_sft_data.jsonl",
    "drug_evidence_grader": "datasets/agent_sft/drug_evidence_grader/drug_evidence_grader_sft_data.jsonl",
    "diagnosis_generator": "datasets/agent_sft/diagnosis_generator/diagnosis_dapt.jsonl",
}

OUTPUT_BY_AGENT = {
    "symptom_normalizer": "models/adapters/symptom_normalizer",
    "symptom_quality_grader": "models/adapters/symptom_quality_grader",
    "rag_relevance_grader": "models/adapters/rag_relevance_grader",
    "drug_evidence_grader": "models/adapters/drug_evidence_grader",
    "diagnosis_generator": "models/adapters/diagnosis_generator",
}

assert AGENT_NAME in MODEL_BY_AGENT, f"Unknown agent: {AGENT_NAME}"

MODEL_NAME = MODEL_BY_AGENT[AGENT_NAME]
TRAIN_FILE = DATA_BY_AGENT[AGENT_NAME]
OUTPUT_DIR = OUTPUT_BY_AGENT[AGENT_NAME]

print("AGENT_NAME:", AGENT_NAME)
print("MODEL_NAME:", MODEL_NAME)
print("TRAIN_FILE:", TRAIN_FILE)
print("OUTPUT_DIR:", OUTPUT_DIR)

## 2) Build command

- `diagnosis_generator` uses `pretraining.py` (DAPT/QA).
- Others use `supervised_finetuning.py` (SFT).

In [ ]:
SFT_SCRIPT = Path("../MyMedicalGPT/supervised_finetuning.py")
DAPT_SCRIPT = Path("../MyMedicalGPT/pretraining.py")

EPOCHS = 2
LR = "2e-4"
BATCH = "2"
GRAD_ACC = "8"
LORA_R = "64"
LORA_ALPHA = "128"
LORA_DROPOUT = "0.05"

script = DAPT_SCRIPT if AGENT_NAME == "diagnosis_generator" else SFT_SCRIPT

cmd = [
    "python", str(script),
    "--model_name_or_path", MODEL_NAME,
    "--tokenizer_name_or_path", MODEL_NAME,
    "--train_file", TRAIN_FILE,
    "--output_dir", OUTPUT_DIR,
    "--per_device_train_batch_size", BATCH,
    "--gradient_accumulation_steps", GRAD_ACC,
    "--num_train_epochs", str(1 if AGENT_NAME == "diagnosis_generator" else EPOCHS),
    "--learning_rate", LR,
    "--lora_rank", LORA_R,
    "--lora_alpha", LORA_ALPHA,
    "--lora_dropout", LORA_DROPOUT,
]

print("Command:")
print(" ".join(shlex.quote(x) for x in cmd))

In [ ]:
# Run training
subprocess.run(cmd, check=True)

## 3) Next agent

After this run finishes:
1. Change `AGENT_NAME`
2. Run all cells again

This guarantees one model artifact per agent under `models/adapters/<agent>`